# **Modelling and Evaluation**

## Objectives

* Answer Business Requirement 2:
    * The client is interested in predicting if a cherry leaf is healthy or contains powdery mildew.

## Inputs

* /workspace/mildew-detection-in-cherry-leaves/inputs/cherry-leaves/train
* /workspace/mildew-detection-in-cherry-leaves/inputs/cherry-leaves/validation
* /workspace/mildew-detection-in-cherry-leaves/inputs/cherry-leaves/test
* image shape embeddings.

## Outputs

* Images distribution plot in train, validation and test sets.
* Image augmentation.
* Class indices to change prediction inference in labels.
* Machine learning model creation and training.
* Save model.
* Learning curve plot for model performance.
* Model evaluation on pickle file.
* Prediction on a random image file.

---



---

# Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from matplotlib.image import imread

---

### Set working directory

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
os.chdir(os.path.dirname(cwd))
print("You set a new current directory")

In [ ]:
working_dir = os.getcwd()
working_dir

---

### Set input directories

Set train, validation and test paths

In [5]:
my_data_dir = 'inputs/cherry-leaves'
train_path = os.path.join(my_data_dir, 'train')
val_path = os.path.join(my_data_dir, 'validation')
test_path = os.path.join(my_data_dir, 'test')

### Set output directory

In [ ]:
version = 'v5'
file_path = os.path.join('outputs', version)

if 'outputs' in os.listdir(working_dir) and version in os.listdir(os.path.join(working_dir, 'outputs')):
    print('Old version is already available. Create a new version')
    pass
else:
    os.makedirs(name=file_path)

### Set label names

In [ ]:
labels = os.listdir(train_path)
print('Labels for the images are', labels)

### Set image shape

In [ ]:
import joblib
version = 'v5'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Images Distribution in Train, Validation and Test datasets

### Image Distribution Bar Figure

In [ ]:
df_freq = pd.DataFrame([])
for folder in ['train', 'validation', 'test']:
    for label in labels:
        df_freq = df_freq.append(
            pd.Series(data={'Set': folder,
                            'Label': label,
                            'Frequency': int(len(os.listdir(os.path.join(my_data_dir, folder, label))))}
                      ),
            ignore_index=True
        )

        print(
            f"* {folder} - {label}: {len(os.listdir(os.path.join(my_data_dir, folder, label)))} images")

print("\n")
sns.set_style("whitegrid")
plt.figure(figsize=(6, 4))
plt.title('Cherry Leaves Dataset Label Distribution')
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.savefig(f'{file_path}/labels_distribution.png',
            bbox_inches='tight', dpi=150)
plt.show()

### Image Distribution Pie Figure

In [ ]:
folders = os.listdir(my_data_dir)
data = []

# Iterating through folders and labels
for folder in folders:
    n = 0  # Initialize count
    for label in labels:
        # Get the number of files in each subdirectory
        n += len(os.listdir(os.path.join(my_data_dir, folder, label)))
    data.append(n)

# Plotting the pie chart
px = 1 / plt.rcParams['figure.dpi']
plt.subplots(figsize=(800 * px, 250 * px))
colors = sns.color_palette('deep')[1:6]
plt.pie(data, labels=folders, colors=colors, autopct='%.0f%%')
plt.title('Cherry Leaves Dataset Distribution')
plt.savefig(f'{file_path}/sets_distribution_pie.png', bbox_inches='tight', dpi=150)
plt.show()


---

## Image Data Augmentation

### ImageDataGenerator

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

* ### Initialise ImageDataGenerator

In [19]:
augmented_image_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

* ### Augment training image dataset

In [ ]:
BATCH_SIZE = 20 # set batch size

train_data = augmented_image_data.flow_from_directory(train_path,
                                                      target_size=image_shape[:2],
                                                      color_mode='rgb',
                                                      batch_size=BATCH_SIZE,
                                                      class_mode='categorical',
                                                      shuffle=True
                                                      )

train_data.class_indices

* ### Augment validation image dataset

In [ ]:
validation_data = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                         target_size=image_shape[:2],
                                                                         color_mode='rgb',
                                                                         batch_size=BATCH_SIZE,
                                                                         class_mode='categorical',
                                                                         shuffle=False
                                                                         )

* ### Augment test image dataset

In [ ]:
test_data = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                   target_size=image_shape[:2],
                                                                   color_mode='rgb',
                                                                   batch_size=BATCH_SIZE,
                                                                   class_mode='categorical',
                                                                   shuffle=False
                                                                   )

* ### Plot augmented training image

In [ ]:
for i in range(5):
    img, label = train_data.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

* ### Plot augmented validation and test images

In [ ]:
for i in range(5):
    img, label = validation_data.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for i in range(5):
    img, label = test_data.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

### Save class_indices

In [ ]:
joblib.dump(value=train_data.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---

# Model Creation

---

## ML model

* ### Import model packages


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam
from tensorflow.keras.regularizers import l2

* ### Model

In [28]:
def create_tf_model():
    model = Sequential()

    # First Convolutional Block
    model.add(Conv2D(filters=32, kernel_size=(3, 3),
                     input_shape=image_shape, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Block
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Convolutional Block
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.6))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad',
                  metrics=['accuracy'])

    return model


* ### Model Summary

In [ ]:
create_tf_model().summary()

* ### Early Stopping/Callbacks

In [30]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# m_checkpoint = ModelCheckpoint(
#     filepath='outputs/v4/mildew_detection_model_{epoch:02d}_{val_accuracy:.4f}.h5',  # Save with epoch and val_accuracy in the filename
#     monitor='val_accuracy',  # Monitor validation accuracy
#     mode='max',  # Save when accuracy improves
#     save_best_only=True,  # Only save the best model to avoid multiple files
#     save_weights_only=False,  # Save the entire model (architecture + weights)
#     verbose=1  # Verbosity to show when saving happens
# )


### Fit model for model training

In [ ]:
model = create_tf_model()
history = model.fit(train_data,
          epochs=25,
          steps_per_epoch=len(train_data.classes) // BATCH_SIZE,
          validation_data=validation_data,
          callbacks=[early_stop, reduce_lr],
          verbose=1
          )

### Save model

In [32]:
model.save('outputs/v5/mildew_detection_model.h5')

---

## Model Performance

---

### Model Learning Curve - A

In [ ]:
losses = pd.DataFrame(model.history.history)

sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses.png',
            bbox_inches='tight', dpi=150)
plt.show()

print("\n")
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_accuracy.png',
            bbox_inches='tight', dpi=150)
plt.show()

### Model Learning Curve - B

In [ ]:
pd.DataFrame(model.history.history).plot(figsize=(8, 5))

plt.savefig(f'{file_path}/model_loss_accuracy.png',
            bbox_inches='tight', dpi=150)

plt.show()

### Model Learning Curve - C

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter( y=model.history.history['val_loss'], name="val_loss"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter( y=model.history.history['loss'], name="loss"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter( y=model.history.history['val_accuracy'], name="val accuracy"),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter( y=model.history.history['accuracy'], name="accuracy"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Loss/Accuracy of LSTM Model"
)

fig.update_xaxes(title_text="Epoch")

fig.update_yaxes(title_text="<b>primary</b> Loss", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Accuracy", secondary_y=True)

fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )

fig.show()
fig.write_image(f'{file_path}/model_history.png')

---

## Model Evaluation

### Import Packages

In [9]:
import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

### Load saved model

In [12]:
from keras.models import load_model
model = load_model('outputs/v5/mildew_detection_model.h5')

### Accuracy
Aims:
* Evaluation of the model on the test set.
* Confirm that the performance on the test set meets the requirements of at least 65% accuracy and answers Business Requirement 2.

In [ ]:
evaluation = model.evaluate(test_data, batch_size=BATCH_SIZE)
print("Model Accuracy: {:.2f}%".format(evaluation[1] * 100))
print("Model Loss: ", evaluation[0])

### Set accuracy variables

In [40]:
validation_data.reset()

x_true, y_true = next(test_data)
preds = np.argmax(model.predict(test_data), axis=1)
y_pred = np.rint(preds)
y_true = test_data.labels

### ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score

sns.set_style("whitegrid")
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 5))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, label="Random Classifier", linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig(f'{file_path}/roccurve.png',
            bbox_inches='tight', dpi=150)
print('Area Under ROC-Curve: ', roc_auc_score(y_true, y_pred))
plt.show()

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true,y_pred)

classes=list(test_data.class_indices.keys()) 
length=len(classes)
plt.figure(figsize=(8, 5))
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
plt.xticks(np.arange(length)+.5, classes, rotation= 0, fontsize=8)
plt.yticks(np.arange(length)+.3, classes, rotation=90, fontsize=8)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig(f'{file_path}/confusion_matrix.png',
            bbox_inches='tight', dpi=150)
plt.show()

### Classification Report - A

In [7]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
print('Classification Report A:\n----------------------\n')
print(classification_report(y_true, y_pred, target_names=labels))

### Classification Report - B

In [ ]:
import matplotlib as mpl
clf_report = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
fig, ax = plt.subplots(figsize=(8,5))
sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True, cmap="Blues", cbar=False, linewidths=1)
plt.title('Classification Report B')

### Classification Report - C

In [ ]:
import matplotlib.pyplot as plt
import pathlib


def plot_classification_report(y_true, y_pred, title='Classification Report', save_fig_path=None, **kwargs):
    # Create classification report dictionary
    clf_report = classification_report(y_true, y_pred, output_dict=True, **kwargs)

    # Filter out 'accuracy', 'macro avg', and 'weighted avg'
    keys_to_plot = [key for key in clf_report.keys() if key not in ('accuracy', 'macro avg', 'weighted avg')]
    df = pd.DataFrame(clf_report, columns=keys_to_plot).T
    
    # Sort by 'support' column
    df.sort_values(by=['support'], inplace=True)

    # Create a mask for the support column to handle it separately
    rows, cols = df.shape
    mask = np.zeros((rows, cols))
    mask[:, cols-1] = True  # Mask out the 'support' column for the first heatmap

    fig, ax = plt.subplots(figsize=(5, 5))

    # Plot the heatmap without 'support'
    ax = sns.heatmap(df.iloc[:, :-1], mask=mask[:, :-1], annot=True, cmap='plasma', fmt='.3g', cbar=False,
                     vmin=0.0, vmax=1.0, linewidths=.4, linecolor='white')

    # Mask for 'support' column (second heatmap)
    mask_support = np.zeros((rows, cols))
    mask_support[:, :-1] = True  # Mask everything except 'support'

    # Plot support column as a second heatmap with separate formatting
    ax = sns.heatmap(df.iloc[:, -1:], mask=mask_support[:, -1:], annot=True, cmap='YlGn', fmt=',.0f', cbar=False,
                     linewidths=2, linecolor='white', 
                     vmin=df['support'].min(), vmax=df['support'].sum(),
                     norm=mpl.colors.Normalize(vmin=df['support'].min(), vmax=df['support'].sum())
                    )
    
    # Set the title and axis labels
    plt.title(title)
    plt.yticks(np.arange(len(df)) + 0.5, df.index, rotation=0)  # Fix the y-tick labels

    # Save the figure if a path is provided
    if save_fig_path:
        path = pathlib.Path(save_fig_path)
        path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_fig_path)
    
    return fig, ax


fig, ax = plot_classification_report(y_true, y_pred, 
                                     title='Classification Report',
                                     target_names=labels,
                                     save_fig_path=f'{file_path}/clf_report.png')


### Save Evaluation Pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v5/evaluation.pkl")

---

## Predict on New Data

### Load Random Image as PIL

In [ ]:
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Assuming test_path, labels, and image_shape are defined earlier in the code
pointer = 44  # Choose a specific image index
label = labels[0]  # '1' corresponds to 'powdery mildew', '0' would be for 'healthy'

# Load the image using Keras' image processing utilities
img_path = os.path.join(test_path, label, os.listdir(os.path.join(test_path, label))[pointer])
pil_image = image.load_img(img_path, target_size=image_shape, color_mode='rgb')

# Print image details
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')

# Display the image
plt.imshow(pil_image)
plt.axis('off')  # Hide the axis
plt.show()


### Convert Image to Array for Prediction

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255
print(my_image.shape)

### Predict Class Probabilities

In [ ]:
pred_proba = model.predict(my_image)[0, 0]

target_map = {v: k for k, v in train_data.class_indices.items()}
pred_class = target_map[pred_proba < 0.5]

if pred_class == target_map[1]:
    pred_proba = 1 - pred_proba

print(f"{pred_class} {round(pred_proba*100, 2)}%")
